# 1) Introduction


<h2> Covered in the previous notebook </h2>
<br>
<br>
<br>

# 2) Project Management

<h2> Covered in the previous notebook </h2>
<br>
<br>
<br>

# 3) Env Setup

In [ ]:
!pip install langchain==0.2.3 langchain_experimental umap-learn scikit-learn langchain_community tiktoken langchain-openai langchainhub chromadb langchain-anthropic wikipedia
!pip install beautifulsoup4 lxml # for scraping and parsing

In [ ]:
!pip install -q streamlit
!npm install localtunnel

In [ ]:
%%writefile requirements_Inventures_RAG_with_UI.txt
# pip freeze on Google Colab

Writing requirements_Inventures_RAG_with_UI.txt


In [ ]:
!pip freeze >> requirements_Inventures_RAG.txt

In [4]:
%%writefile mylibs.py
# Importing libraries
import os
import pickle
import json # saving sessions data in a json file
from google.colab import userdata # for importing env variable on Colab
import matplotlib.pyplot as plt
import tiktoken # for counting the # of tokens in each document
import numpy as np
import pandas as pd
import umap # for dimensionality reduction
from sklearn.mixture import GaussianMixture # for clustering
from typing import Dict, List, Optional, Tuple # for type hinting

from bs4 import BeautifulSoup # for scraping and parsing

# Wikiloader in case we want to retrieve data from Wikipedia as a trainset
from langchain_community.document_loaders import WikipediaLoader
# converting doc_text files into langchain document objects
from langchain.docstore.document import Document

from langchain_openai import OpenAIEmbeddings # For text-to-vector
from langchain_openai import ChatOpenAI # For API call to LLM
from langchain_anthropic import ChatAnthropic # Alternative to OpenAI LLM

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

RANDOM_SEED = 307  # Fixed seed for reproducibility of clustering

from langchain_community.vectorstores import Chroma #In-memory VectorDB

from langchain import hub # Langchain Hub for RAG prompt templates
# placeholder used in chain
from langchain_core.runnables import RunnablePassthrough
import streamlit as st



In [5]:
# Args and global variables
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = 'anthropic-api-key'

# 4) Data Handeling

## 4.1 Scraping and parsing the data

In [6]:
def parse_sessions_html(html_file_add):
  # Read the HTML file
  with open(html_file_add, 'r', encoding='utf-8') as file:
    html_content = file.read()

  # Parse the HTML content using BeautifulSoup
  soup = BeautifulSoup(html_content, 'lxml')

  # Extract session details
  sessions = []
  session_divs = soup.find_all('div', attrs={"x-html": "session.card_template"})

  for session_div in session_divs:
    session = {}

    # Extract session category
    category_tag = session_div.find('span', class_="block w-fit bg-green py-2 -mt-1 md:ml-0 px-6 lg:pr-8 text-xs font-moderna rounded-br-full")
    session['category'] = category_tag.text.strip() if category_tag else None

    # Extract session title
    title_tag = session_div.find('h2', class_="font-moderna uppercase text-base xl:text-lg text-purple")
    session['title'] = title_tag.text.strip() if title_tag else None

    # Extract session description
    description_tag = session_div.find('div', class_="cursor-pointer text-base line-clamp-2")
    session['description'] = description_tag.text.strip() if description_tag else None

    # Extract speakers
    speakers = []
    speaker_section = session_div.find('div', class_="md:border-t border-slate-200 md:pt-4")
    if speaker_section:
      speaker_buttons = speaker_section.find_all('button', class_="group relative flex gap-4 justify-start items-center !no-underline")
      for speaker_button in speaker_buttons:
        speaker = {}

        # Extract speaker name
        name_tag = speaker_button.find('h4', class_="text-sm font-moderna")
        speaker['name'] = name_tag.text.strip() if name_tag else None

        # Extract speaker position
        position_tag = speaker_button.find('p', class_="override text-sm leading-tight text-slate-500")
        speaker['position'] = position_tag.text.strip() if position_tag else None

        speakers.append(speaker)

    session['speakers'] = speakers
    sessions.append(session)

  sessions.pop(0) # The first session is empty
  # Convert the sessions list to a JSON string
  json_string = json.dumps(sessions)
  # Write the JSON string to a file
  with open('sessions.json', 'w') as f:
    f.write(json_string)

  return sessions


## 4.2) Processing data

In [7]:
def save_sessions_to_text_files(sessions):
    """Save sessions to a separate text files."""
    for i, session in enumerate(sessions, start=1):
      category = session['category']
      title = session['title']
      description = session['description']
      speakers = session['speakers']
      content = f"A session under the category of : {category} which is titled as: {title} is held at Inventures. The description of this session is: {description}. \n "

      if speakers:
        content += "The speakers of this session are: \n"
      for speaker in speakers:
        name = speaker['name']
        position = speaker['position']
        content += f"  - Name: {name} who has a position: {position}\n"

      content += "\n"

      with open(f"./sessions_txt/{i}.txt", "a", encoding='utf-8') as file:
        file.write(content)
        file.close()

# 5) Modeling

In [8]:
# Reference for this cell:
# https://github.com/langchain-ai/langchain/blob/master/cookbook/

### --- Code from citations referenced above (added comments and docstrings) --- ###


def global_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """
    Perform global dimensionality reduction on the embeddings using UMAP.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - n_neighbors: Optional; the number of neighbors to consider for each point.
                   If not provided, it defaults to the square root of the number of embeddings.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def local_cluster_embeddings(
    embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
) -> np.ndarray:
    """
    Perform local dimensionality reduction on the embeddings using UMAP, typically after global clustering.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - num_neighbors: The number of neighbors to consider for each point.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    return umap.UMAP(
        n_neighbors=num_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def get_optimal_clusters(
    embeddings: np.ndarray, max_clusters: int = 50, random_state: int = RANDOM_SEED
) -> int:
    """
    Determine the optimal number of clusters using the Bayesian Information Criterion (BIC) with a Gaussian Mixture Model.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - max_clusters: The maximum number of clusters to consider.
    - random_state: Seed for reproducibility.

    Returns:
    - An integer representing the optimal number of clusters found.
    """
    max_clusters = min(max_clusters, len(embeddings))
    n_clusters = np.arange(1, max_clusters)
    bics = []
    for n in n_clusters:
        gm = GaussianMixture(n_components=n, random_state=random_state)
        gm.fit(embeddings)
        bics.append(gm.bic(embeddings))
    return n_clusters[np.argmin(bics)]


def GMM_cluster(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """
    Cluster embeddings using a Gaussian Mixture Model (GMM) based on a probability threshold.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - threshold: The probability threshold for assigning an embedding to a cluster.
    - random_state: Seed for reproducibility.

    Returns:
    - A tuple containing the cluster labels and the number of clusters determined.
    """
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state)
    gm.fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters


def perform_clustering(
    embeddings: np.ndarray,
    dim: int,
    threshold: float,
) -> List[np.ndarray]:
    """
    Perform clustering on the embeddings by first reducing their dimensionality globally, then clustering
    using a Gaussian Mixture Model, and finally performing local clustering within each global cluster.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for UMAP reduction.
    - threshold: The probability threshold for assigning an embedding to a cluster in GMM.

    Returns:
    - A list of numpy arrays, where each array contains the cluster IDs for each embedding.
    """
    if len(embeddings) <= dim + 1:
        # Avoid clustering when there's insufficient data
        return [np.array([0]) for _ in range(len(embeddings))]

    # Global dimensionality reduction
    reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)
    # Global clustering
    global_clusters, n_global_clusters = GMM_cluster(
        reduced_embeddings_global, threshold
    )

    all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
    total_clusters = 0

    # Iterate through each global cluster to perform local clustering
    for i in range(n_global_clusters):
        # Extract embeddings belonging to the current global cluster
        global_cluster_embeddings_ = embeddings[
            np.array([i in gc for gc in global_clusters])
        ]

        if len(global_cluster_embeddings_) == 0:
            continue
        if len(global_cluster_embeddings_) <= dim + 1:
            # Handle small clusters with direct assignment
            local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
            n_local_clusters = 1
        else:
            # Local dimensionality reduction and clustering
            reduced_embeddings_local = local_cluster_embeddings(
                global_cluster_embeddings_, dim
            )
            local_clusters, n_local_clusters = GMM_cluster(
                reduced_embeddings_local, threshold
            )

        # Assign local cluster IDs, adjusting for total clusters already processed
        for j in range(n_local_clusters):
            local_cluster_embeddings_ = global_cluster_embeddings_[
                np.array([j in lc for lc in local_clusters])
            ]
            indices = np.where(
                (embeddings == local_cluster_embeddings_[:, None]).all(-1)
            )[1]
            for idx in indices:
                all_local_clusters[idx] = np.append(
                    all_local_clusters[idx], j + total_clusters
                )

        total_clusters += n_local_clusters

    return all_local_clusters


### --- Our code below --- ###


def embed(texts):
    """
    Generate embeddings for a list of text documents.

    This function assumes the existence of an `embd` object with a method `embed_documents`
    that takes a list of texts and returns their embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be embedded.

    Returns:
    - numpy.ndarray: An array of embeddings for the given text documents.
    """
    text_embeddings = embd.embed_documents(texts)
    text_embeddings_np = np.array(text_embeddings)
    return text_embeddings_np


def embed_cluster_texts(texts):
    """
    Embeds a list of texts and clusters them, returning a DataFrame with texts, their embeddings, and cluster labels.

    This function combines embedding generation and clustering into a single step. It assumes the existence
    of a previously defined `perform_clustering` function that performs clustering on the embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be processed.

    Returns:
    - pandas.DataFrame: A DataFrame containing the original texts, their embeddings, and the assigned cluster labels.
    """
    text_embeddings_np = embed(texts)  # Generate embeddings
    cluster_labels = perform_clustering(
        text_embeddings_np, 10, 0.1
    )  # Perform clustering on the embeddings
    df = pd.DataFrame()  # Initialize a DataFrame to store the results
    df["text"] = texts  # Store original texts
    df["embd"] = list(text_embeddings_np)  # Store embeddings as a list in the DataFrame
    df["cluster"] = cluster_labels  # Store cluster labels
    return df


def fmt_txt(df: pd.DataFrame) -> str:
    """
    Formats the text documents in a DataFrame into a single string.

    Parameters:
    - df: DataFrame containing the 'text' column with text documents to format.

    Returns:
    - A single string where all text documents are joined by a specific delimiter.
    """
    unique_txt = df["text"].tolist()
    return "--- --- \n --- --- ".join(unique_txt)


def embed_cluster_summarize_texts(
    texts: List[str], level: int
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Embeds, clusters, and summarizes a list of texts. This function first generates embeddings for the texts,
    clusters them based on similarity, expands the cluster assignments for easier processing, and then summarizes
    the content within each cluster.

    Parameters:
    - texts: A list of text documents to be processed.
    - level: An integer parameter that could define the depth or detail of processing.

    Returns:
    - Tuple containing two DataFrames:
      1. The first DataFrame (`df_clusters`) includes the original texts, their embeddings, and cluster assignments.
      2. The second DataFrame (`df_summary`) contains summaries for each cluster, the specified level of detail,
         and the cluster identifiers.
    """

    # Embed and cluster the texts, resulting in a DataFrame with 'text', 'embd', and 'cluster' columns
    df_clusters = embed_cluster_texts(texts)

    # Prepare to expand the DataFrame for easier manipulation of clusters
    expanded_list = []

    # Expand DataFrame entries to document-cluster pairings for straightforward processing
    for index, row in df_clusters.iterrows():
        for cluster in row["cluster"]:
            expanded_list.append(
                {"text": row["text"], "embd": row["embd"], "cluster": cluster}
            )

    # Create a new DataFrame from the expanded list
    expanded_df = pd.DataFrame(expanded_list)

    # Retrieve unique cluster identifiers for processing
    all_clusters = expanded_df["cluster"].unique()

    print(f"--Generated {len(all_clusters)} clusters--")

    # Summarization
    template = """
    Give a detailed summary of the documentation provided.

    Documentation:
    {context}
    """
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | StrOutputParser()

    # Format text within each cluster for summarization
    summaries = []
    for i in all_clusters:
        df_cluster = expanded_df[expanded_df["cluster"] == i]
        formatted_txt = fmt_txt(df_cluster)
        summaries.append(chain.invoke({"context": formatted_txt}))

    # Create a DataFrame to store summaries with their corresponding cluster and level
    df_summary = pd.DataFrame(
        {
            "summaries": summaries,
            "level": [level] * len(summaries),
            "cluster": list(all_clusters),
        }
    )

    return df_clusters, df_summary


def recursive_embed_cluster_summarize(
    texts: List[str], level: int = 1, n_levels: int = 3
) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    Recursively embeds, clusters, and summarizes texts up to a specified level or until
    the number of unique clusters becomes 1, storing the results at each level.

    Parameters:
    - texts: List[str], texts to be processed.
    - level: int, current recursion level (starts at 1).
    - n_levels: int, maximum depth of recursion.

    Returns:
    - Dict[int, Tuple[pd.DataFrame, pd.DataFrame]], a dictionary where keys are the recursion
      levels and values are tuples containing the clusters DataFrame and summaries DataFrame at that level.
    """
    results = {}  # Dictionary to store results at each level

    # Perform embedding, clustering, and summarization for the current level
    df_clusters, df_summary = embed_cluster_summarize_texts(texts, level)

    # Store the results of the current level
    results[level] = (df_clusters, df_summary)

    # Determine if further recursion is possible and meaningful
    unique_clusters = df_summary["cluster"].nunique()
    if level < n_levels and unique_clusters > 1:
        # Use summaries as the input texts for the next level of recursion
        new_texts = df_summary["summaries"].tolist()
        next_level_results = recursive_embed_cluster_summarize(
            new_texts, level + 1, n_levels
        )

        # Merge the results from the next level into the current results dictionary
        results.update(next_level_results)

    return results


# 6) Evaluation

In [ ]:
# We're using human evaluation on different aspects of automatic text
# summarization and question answering

# 7) Deployment & Maintnance

In [12]:
%%capture
!unzip sessions_txt.zip

In [22]:
%%writefile app.py
#initial files
import streamlit as st

st.title("RAG Bot")
with st.chat_message("assistant"):
  st.write("Hello! Please wait till I spin up 🔄 ...")

if 'initialized' not in st.session_state:
  # python does not have block-level scope within conditional
  # statements (if, for, while, etc.). Instead, Python uses function-level
  # scope. This means that variables defined inside an if statement
  # are still accessible outside of that statement, provided they are
  # within the same function or global scope. But, streamlit only remembers
  # the st.session_state in subsequent runs. That's why we need to store
  # rag_chain in state
  from mylibs import *
  st.session_state.initialized = True
  # Initialize chat history
  if "messages" not in st.session_state:
    st.session_state.messages = []

  doc_text = [] # loading the txt files into a list of strings
  for i in range(1,187):
    with open(f'./sessions_txt/{i}.txt', 'r') as file:
      data = file.read()
      doc_text.append(data)

  raw_documents = []
  for i in range(len(doc_text)):
    raw_documents.append(Document(page_content=doc_text[i], metadata={"source": "local", 'id':i+1}))

  embd = OpenAIEmbeddings()
  model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=4096)
  leaf_texts = doc_text
  # unpickle results.pkl into results
  with open("results.pkl", "rb") as f:
      results = pickle.load(f)
  # Initialize all_texts with leaf_texts
  all_texts = leaf_texts.copy()

  # Iterate through the results to extract summaries from each level and add them to all_texts
  for level in sorted(results.keys()):
    # Extract summaries from the current level's DataFrame
    summaries = results[level][1]["summaries"].tolist()
    # Extend all_texts with the summaries from the current level
    all_texts.extend(summaries)

  # Now, use all_texts to build the vectorstore with Chroma
  vectorstore = Chroma.from_texts(texts=all_texts, embedding=embd)
  retriever = vectorstore.as_retriever()

  # Prompt
  prompt = hub.pull("rlm/rag-prompt")


  # Post-processing (joining all docs retrieved after vector search)
  def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

  # Chain
  st.session_state.rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | model
      | StrOutputParser()
  )
else:
  # Retrieve the chain from session state
  rag_chain = st.session_state.rag_chain


# Display chat messages from history on app return
for message in st.session_state.messages:
    with st.chat_message(message['role']):
        st.markdown(message['content'])


# React to user input
if prompt := st.chat_input("What's up?"):
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)
    # Add user message to chat history
    st.session_state.messages.append({'role': "user", "content": prompt})

    # Invoke the RAG chain to get the response
    ans = rag_chain.invoke(prompt)

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        st.markdown(ans)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant",
                                      "content": ans})


Overwriting app.py


In [23]:
# Start the app locally
!streamlit run app.py &>/content/logs.txt & curl https://loca.lt/mytunnelpassword

34.106.176.117

In [24]:
# Expose the app to the world
!npx localtunnel --port 8501

npx: installed 22 in 2.275s
your url is: https://good-papers-stand.loca.lt
^C


# 8) Conclusion

In [19]:
query = "Tell me about the topics discussed at Inventures"

<pr> Let's review our work through a lense that I learnt recently from, which is <strong> Problem-Solution-Benefits </strong> </pr>
<br>

<h2>Problem; Beat it or join it! </h2>

<br>
<h2>Solution; We implemented RAG yet can go further! </h2>

<br>
<h2>Benefits; Data retrieval and analysis never gonna go out of style!</h2>